# Lecture10

The purpose of this assignment is to gain intuition on convolution and how various kernels interact with images. Note: technically we will be using cross-correlation but we will still use the convolution terminolgy. 

In [ ]:
import SimpleITK as sitk
import numpy as np
import torch
import os
import utils
import matplotlib.pyplot as plt

In [ ]:
if torch.backends.mps.is_available():
    print("MPS is available.")
    device = torch.device("mps")
elif torch.cuda.is_available():
    print("CUDA is available.")
    device = torch.device("cuda")
else:
    print("Using CPU")
    device = torch.device("cpu")


engr_dir = "/opt/nfsopt/DLMI"
idas_dir = os.path.join(os.path.expanduser('~'), "classdata")

if os.path.isdir(engr_dir):
    data_dir = engr_dir
elif os.path.isdir(idas_dir):  
    data_dir = idas_dir
else:
    print("Data directory not found")

## Part 1

A brain MRI image will be used in this assingment. First load the image:

In [ ]:
fn = os.path.join(data_dir, "MRHead.nii.gz")

im = sitk.ReadImage(fn)

Next view the image, using the `myshow_mutliview` function defined in the `utils.py` file. This function assumes images have identity direction cosine matrix. The original image does not have identity direction cosine matrix, so first we will standardize the orientation using the `standardize_orientation` function defined in `utils.py` 

In [ ]:
im_stand = utils.standardize_orientation(im)
utils.myshow_multiview(im_stand, [40,140,160], 0, 100)

print("Image Size: {}".format(im_stand.GetSize()))
print("Image Spacing: {}".format(im_stand.GetSpacing()))
print("Image Direction: {}".format(im_stand.GetDirection()))

Next we will perform image convolution between the image and a hand crafted kernel. For efficiency and interpretability, we will perform 2D convolution on individual slices, rather than 3D convolution.  

We will use the PyTorch layer called `torch.nn.Conv2d` to perform 2D image convolution. The PyTorch implementation assumes the input image and kernel have rank 4 (batch, channel, spatial, spatial) - we will learn more about the first two dimensions in future lectures, for now we will just add 2 dummy dimensions (dimensions with size 1) to our 2D image. Similarly, we will need to add 2 dummy dimensions to our hand crafted kernels. The `myconv2d` function defined below handles this conversion to proper rank, creates an instance of `torch.nn.Conv2d`, and performs convolution.


In [ ]:
def myconv2d(im, kernel):
    """
    Performs image convolution. 

    The input image must be a 2D matrix.

    Inputs:
    - im (np.array): Input image represented as a numpy array with rank 2
    - kernel (np.array): Kernel represented as a numpy array with rank 2

    Returns:
    - act_np (np.array): Result of convolving the im with kernel.
    """
    # to float 32
    im = im.astype(np.float32)
    kernel = kernel.astype(np.float32)
    
    # convert to pytorch tensor
    im_tensor = torch.from_numpy(im).to(device)
    kernel_tensor = torch.from_numpy(kernel).to(device)
    
    # add 2 dummy dims to create rank 4 tensors
    im_tensor = im_tensor.unsqueeze(0).unsqueeze(0)
    kernel_tensor = kernel_tensor.unsqueeze(0).unsqueeze(0)

    # create instance of conv layer
    conv_layer = torch.nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, bias=False)

    # note the below is specific to using the conv2d layer with fixed hand-crafted kernels
    # this will not typically be done when training a CNN
    with torch.no_grad():
        conv_layer.weight.copy_(kernel_tensor)
    conv_layer.weight.requires_grad = False
    
    # peforms convolution (calls forward method)
    act = conv_layer(im_tensor)
    
    # remove dummy dims, back to numpy
    act_np = act.squeeze().cpu().detach().numpy()

    return act_np

<br>
 
Create a handcrafted kernel and convolve it with the brain MRI image. First, use the following $3\times3$ kernel:

\begin{matrix}
1 & 2 & 1\\
0 & 0 & 0\\
-1 & -2 & -1
\end{matrix}

Create the kernel as a numpy array, and visualize it using `imshow`:

In [ ]:
kernel = np.array([[1,2,1],
                   [0,0,0],
                   [-1,-2,-1]])

fig, ax = plt.subplots(figsize = (2,2))
ax.imshow(kernel, cmap=plt.cm.RdBu)
ax.axis('off')

<br>

Predict what will the output look like if the above kernel is convolved (cross-correlated) with the brain MRI image?

What types of features will be correlated? Negatively correlated?

XXX

Pick a slice and perfom the convolution by calling the `myconv2d` function. You can try different slices by changing the `slc` value (make sure it is a valid slice).

In [ ]:
slc = 160
imnp = sitk.GetArrayFromImage(im_stand)
imnp_ax = imnp[slc,:,:]
im_conv = myconv2d(imnp_ax, kernel)

Visualize the output of the convolution below:

In [ ]:
fig, axs = plt.subplots(1,2, figsize = (8,4))
spac = im_stand.GetSpacing()
size = im_stand.GetSize()
im_disp = axs[0].imshow(imnp_ax, cmap='gray', aspect = spac[1]/spac[0], vmin = 0, vmax = 100)
out_disp = axs[1].imshow(im_conv, cmap=plt.cm.RdBu, aspect = spac[1]/spac[0])
fig.colorbar(im_disp, label="Intensity",ax = axs[0])
fig.colorbar(out_disp, label="Intensity",ax = axs[1])
for ax in axs:
    ax.axis('off')

<br>

Discuss what you observe. What image features show the strongest positive correlation? The strongest negative correlation? No correlation?

XXX

## Part 2

Next, load the `MRHead_conv_slc_160.npy` array and visualize using `imshow`.

In [ ]:
fn = os.path.join(data_dir, "MRHead_conv_slc_160.npy")

im_mystery = np.load(fn)

fig, ax = plt.subplots(figsize = (4,4))
disp = ax.imshow(im_mystery, cmap=plt.cm.RdBu, aspect = spac[1]/spac[0])
fig.colorbar(disp, label="Intensity",ax = ax)
ax.axis('off')

The above image is the result of convolving an axial slice (slice index 160) of the brain MRI image with an unknown $3\times3$ kernel. Next, you will try to reproduce the above image by designing a kernel and convolving it with the brain MRI image. The `mse` function (defined in `utils.py`) will be used to quantify the mean squared error between your convolved image and the target image (`im_mystery`). You will need to design the filter, visualize the filter, convolve the kernel with the image, visualize the result, compute the error with your image and the target image. 


In [ ]:
######################################
#######         TODO           #######
######################################

# create and visualize the kernel


In [ ]:
######################################
#######         TODO           #######
######################################

# convolve with brain MRI and visualize result


In [ ]:
error = utils.mse(im_mystery, im_test)
print("MSE: {:.2f}".format(error))

Modify your kernel, and repeat until you are satisfied with your error. For reference, `im_conv` (the image produced in part 1) and `im_mystery` have a MSE of 15519.02, try to get a better MSE than this. 

**Foreshadow:** when trainng a convolutional neural network, the kernel values are generally not explicitly set (hand-crafted) like in this assignment. Rather, the kernel values are the **learnable parameters** that are initalized randomly and optimized through training. 

Save a PDF of the document to upload to ICON and then add/commit/push to your git repository.